In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q streamlit
!pip install geopandas
!pip install folium

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
import os
import pandas as pd
import numpy as np

import geopandas as gpd
import folium

In [ ]:
def new_folder(new_folder_path):
    # Check if the folder exists, if not, create it
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
        print(f'Created new folder at {new_folder_path}')
    else:
        print(f'Folder {new_folder_path} already exists')

In [ ]:
def save_group_to_csv(df,column_name, output_directory,data_type):   
    groups = df.groupby(column_name)
    for name,group in groups:
        filename = f"{output_directory}/{data_type}_{name}.csv"
        group.to_csv(filename, index=False)
        print(f"Exported {filename}")

**0. Départements**

In [ ]:
# Define the path for the new folder
dpt_folder_path = '/kaggle/working/dpt/'
new_folder(dpt_folder_path)

data_dpt_folder_path = '/kaggle/working/dpt/data/'
new_folder(data_dpt_folder_path)

In [ ]:
df_dpt_resultats = pd.read_csv('/kaggle/input/d/jcabouat/resultats-legislatives/resultats-provisoires-par-departement.csv', sep=';')
df_dpt_resultats.rename(columns={'Code département': 'id_dep'}, inplace=True)
df_dpt_resultats.rename(columns={'Libellé département': 'libDepartement'}, inplace=True)
df_dpt_resultats['id_dep'] = df_dpt_resultats['id_dep'].astype(str)
df_dpt_resultats

In [ ]:
df_dpt_resultats = df_dpt_resultats.melt(id_vars=['id_dep', 'libDepartement', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants'])
df_dpt_resultats.rename(columns={'variable': 'indicateur', 'value': 'valeur'}, inplace=True)
df_dpt_resultats_test = df_dpt_resultats[df_dpt_resultats['id_dep'].isin(['14','43','78'])]
df_dpt_resultats_test

In [ ]:
save_group_to_csv(df_dpt_resultats_test,'id_dep','/kaggle/working/dpt/data','resultats')

**1. Circonscriptions législatives**

In [ ]:
# Define the path for the new folder
circo_folder_path = '/kaggle/working/circo/'
new_folder(circo_folder_path)

map_circo_folder_path = '/kaggle/working/circo/map/'
new_folder(map_circo_folder_path)

data_circo_folder_path = '/kaggle/working/circo/data/'
new_folder(data_circo_folder_path)

a. Cartes

In [ ]:
# Set filepath
fp_circo = "/kaggle/input/circonscriptions-legislatives-030522/circonscriptions_legislatives_030522.shp"

# Read file using gpd.read_file()
df_circo = gpd.read_file(fp_circo)
df_circo = df_circo.to_crs(epsg=4326)
df_circo['centroid'] = df_circo.geometry.centroid
print(df_circo.crs)
df_circo.rename(columns={'dep': 'id_dep', 'libelle': 'libCirco' }, inplace=True)
df_circo['id_dep'] = df_circo['id_dep'].astype(str)
df_circo.head()

In [ ]:
# Centroid column
df_circo['lat'] = df_circo["centroid"].y
df_circo['lon'] = df_circo["centroid"].x

df_circo['id_circo_len'] = df_circo['id_circo'].str.len()
df_circo['id_circo'] = np.where(df_circo['id_circo_len'] == 5, df_circo['id_circo'], df_circo['id_circo'].astype(str).str[:2] + '0' + df_circo['id_circo'].astype(str).str[2:])

In [ ]:
df_circo_test = df_circo[df_circo['id_dep'].isin(['14','43','78'])]
df_circo_test

In [ ]:
groups = df_circo_test.groupby('id_dep')
for name,group in groups:
    m = folium.Map(location=[48.858885,2.34694], zoom_start=6, tiles="CartoDB positron")
    for _, r in group.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "blue"})
        folium.Popup(r["libCirco"]+ " " + r["id_circo"]).add_to(geo_j)
        geo_j.add_to(m)
    m.save('/kaggle/working/circo/map/map_' +  r['id_dep'] + '.html')

b. data - https://www.insee.fr/fr/statistiques/6436476?sommaire=6436478

In [ ]:
df_circo_stats = pd.read_excel('/kaggle/input/indic-stat-circo/indic-stat-circonscriptions-legislatives-2022.xlsx',
                               sheet_name='indicateurs_circonscriptions',
                               skiprows=7)
df_circo_stats.rename(columns={'circo': 'id_circo'}, inplace=True)
df_circo_stats['id_dep']= df_circo_stats['id_circo'].str[:2]                                        

In [ ]:
df_circo_stats_test = df_circo_stats[df_circo_stats['id_circo'].str[:2].isin(['14','43','78'])]
df_circo_stats_test.head()

In [ ]:
save_group_to_csv(df_circo_stats_test,'id_dep','/kaggle/working/circo/data','stats')

In [ ]:
df_circo_resultats = pd.read_csv('/kaggle/input/d/jcabouat/resultats-legislatives/resultats-provisoires-par-circonscription.csv', sep=';')
df_circo_resultats.head()

In [ ]:
df_circo_resultats.rename(columns={'Code département': 'id_dep',
                                   'Code circonscription législative': 'id_circo',
                                   'Libellé département': 'libDep',
                                   'Libellé circonscription législative': 'libCirco'
                                  }, inplace=True)
df_circo_resultats['id_dep'] = df_circo_resultats['id_dep'].astype(str)
df_circo_resultats['id_circo'] = df_circo_resultats['id_circo'].astype(str)

df_circo_resultats['id_circo'] = np.where(df_circo_resultats['id_dep'].str.len() == 3,
                             df_circo_resultats['id_dep'] + df_circo_resultats['id_circo'].astype(str).str[2:],
                             df_circo_resultats['id_circo'].astype(str).str[:2] + '0' + df_circo_resultats['id_circo'].astype(str).str[2:])

df_circo_resultats = df_circo_resultats.melt(id_vars=['id_dep', 'libDep','id_circo','libCirco', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants'])
df_circo_resultats.rename(columns={'variable': 'indicateur', 'value': 'valeur'}, inplace=True)

df_circo_resultats_test = df_circo_resultats[df_circo_resultats['id_dep'].isin(['14','43','78'])]
df_circo_resultats_test

In [ ]:
save_group_to_csv(df_circo_resultats_test,'id_dep','/kaggle/working/circo/data','resultats')

**2. Bureaux de votes**

In [ ]:
# Define the path for the new folder
bv_folder_path = '/kaggle/working/bv/'
new_folder(bv_folder_path)

map_bv_folder_path = '/kaggle/working/bv/map/'
new_folder(map_bv_folder_path)

map_bv_folder_path = '/kaggle/working/bv/data/'
new_folder(map_bv_folder_path)

a. Map

In [ ]:
# Set filepath
fp_bv = "/kaggle/input/contours-france-entiere-latest-v2/contours-france-entiere-latest-v2.geojson"

# Read file using gpd.read_file()
df_bv = gpd.read_file(fp_bv)
df_bv = df_bv.to_crs(epsg=4326)
print(df_bv.crs)
df_bv.head()

In [ ]:
df_bv.rename(columns={'codeDepartement': 'id_dep',
                      'codeCirconscription': 'id_circo',
                      'nomDepartement': 'libDep',
                      'nomCirconscription': 'libCirco'
                      }, inplace=True)

df_bv['id_dep'] = df_bv['id_dep'].astype(str)
df_bv['id_circo'] = df_bv['id_circo'].astype(str)
df_bv['id_bv'] = df_bv['id_bv'].astype(str)

df_bv['id_dep'] = np.where(df_bv['id_dep'] == 'ZA', '971',
                           np.where(df_bv['id_dep'] == 'ZB', '972',
                           np.where(df_bv['id_dep'] == 'ZC', '973',
                           np.where(df_bv['id_dep'] == 'ZD', '974',
                           np.where(df_bv['id_dep'] == 'ZS', '975',
                           np.where(df_bv['id_dep'] == 'ZM', '976', df_bv['id_dep']))))))


df_bv['id_circo'] = np.where(df_bv['id_dep'].str.len() == 3,
                             df_bv['id_dep'] + df_bv['id_circo'].astype(str).str[2:],
                             df_bv['id_circo'].astype(str).str[:2] + '0' + df_bv['id_circo'].astype(str).str[2:])

df_bv['libBv'] = df_bv[['nomCommune', 'numeroBureauVote']].agg(' - '.join, axis=1)

df_bv.head()

In [ ]:
df_bv_test = df_bv[df_bv['id_dep'].isin(['14','43','78'])]
export_test = df_bv_test[['id_dep','libDep','id_circo','libCirco','codeCommune','nomCommune','numeroBureauVote','codeBureauVote','id_bv','libBv']]
export_test.to_csv('/kaggle/working/dataset_dpt_circo_bv_test.csv', index=False)

In [ ]:
df_bv_test['centroid'] = df_bv_test.geometry.centroid
df_bv_test['lat'] = df_bv_test["centroid"].y
df_bv_test['lon'] = df_bv_test["centroid"].x

df_bv_test.head()

In [ ]:
groups = df_bv_test.groupby('id_circo')
for name,group in groups:
    m = folium.Map(location=[48.858885,2.34694], zoom_start=6, tiles="CartoDB positron")
    for _, r in group.iterrows():
        # Without simplifying the representation of each borough,
        # the map might not be displayed
        sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "white"})
        folium.Popup(r["libBv"]).add_to(geo_j)
        geo_j.add_to(m)
    m.save('/kaggle/working/bv/map/map_' +  r['id_circo'] + '.html')

b. Résultats Europénnes

In [ ]:
df_bv_resultats = pd.read_csv('/kaggle/input/d/jcabouat/resultats-legislatives/resultats-provisoires-par-bureau-de-votevmn.csv',sep=';',low_memory=False)
df_bv_resultats.head()

In [ ]:
df_bv_resultats.rename(columns={'Code département': 'id_dep',
                      'Libellé département': 'libDep',
                      'Code commune': 'id_com',
                      'Libellé commune': 'libCom',
                      'Code BV': 'codeBV'
                      }, inplace=True)

df_bv_resultats['id_dep'] = df_bv_resultats['id_dep'].astype(str)
df_bv_resultats['libCom'] = df_bv_resultats['libCom'].astype(str)
df_bv_resultats['codeBV'] = df_bv_resultats['codeBV'].astype(str)

df_bv_resultats['libBv'] = df_bv_resultats[['libCom', 'codeBV']].agg(' - '.join, axis=1)

df_bv_resultats['codeBureauVote'] = df_bv_resultats['id_com'].str.cat(df_bv_resultats['codeBV'], sep = '_')

df_bv_resultats.head()

In [ ]:
df_bv_resultats = pd.merge(df_bv_resultats, df_bv[['codeBureauVote','id_circo']], on='codeBureauVote', how='left')
df_bv_resultats = df_bv_resultats.drop_duplicates()
df_bv_resultats

In [ ]:
df_bv_resultats.rename(columns={'codeBureauVote': 'id_bv'}, inplace=True)

In [ ]:
df_bv_resultats_test = df_bv_resultats[df_bv_resultats['id_dep'].isin(['14','43','78'])]
df_bv_resultats_test = df_bv_resultats_test.melt(id_vars=['id_dep','libDep','id_circo','id_com','libCom','codeBV','id_bv','libBv', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants'])
df_bv_resultats_test.rename(columns={'variable': 'indicateur', 'value': 'valeur'}, inplace=True)
df_bv_resultats_test

In [ ]:
save_group_to_csv(df_bv_resultats_test,'id_circo','/kaggle/working/bv/data','resultats')

In [ ]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory_test.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [ ]:
zip_dir()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import streamlit as st
st.set_page_config(layout="wide")

# Load the HTML file
def read_html_file(filename):
    with open(filename, 'r') as f:
        return f.read()


# Stats definitions
df_stats_def = pd.read_csv('/kaggle/input/stats-def-csv/stats_def.csv',sep=';')
df_stats_def_demo = df_stats_def[(df_stats_def['Catégorie'] == 'Démographie') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_acti = df_stats_def[(df_stats_def['Catégorie'] == 'Activité') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_qualif = df_stats_def[(df_stats_def['Catégorie'] == 'Qualification') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_secteur = df_stats_def[(df_stats_def['Catégorie'] == 'Secteur') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_log = df_stats_def[(df_stats_def['Catégorie'] == 'Logements') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_fam = df_stats_def[(df_stats_def['Catégorie'] == 'Famille') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_mob = df_stats_def[(df_stats_def['Catégorie'] == 'Mobilité') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_rev = df_stats_def[(df_stats_def['Catégorie'] == 'Revenus') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_niv_vie = df_stats_def[(df_stats_def['Catégorie'] == 'Niveau de vie') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_autres = df_stats_def[(df_stats_def['Catégorie'] == 'Autres') | (df_stats_def['Variable'] == 'Nom de la circonscription')]


# 1.Liste des départements
file_path = '/kaggle/working/dataset_dpt_circo_bv_test.csv'
df = pd.read_csv(file_path,low_memory=False)
df.rename(columns={'codeDepartement': 'id_dep','nomDepartement': 'dep_name'}, inplace=True)
df['id_dep'] = df['id_dep'].astype(str)
df['id_circo'] = df['id_circo'].astype(str)
df['id_bv'] = df['id_bv'].astype(str)

df_dpt = df.drop(columns=['id_circo','libCirco','codeCommune','nomCommune','numeroBureauVote','codeBureauVote','id_bv','libBv']).drop_duplicates()
df_dpt['id_dep'] = df_dpt['id_dep'].astype(str)
df_dpt.sort_values(by='id_dep',inplace=True)
df_dpt['id_dep'] = df_dpt['id_dep'].astype(str)
df_dpt['dep_lib'] = df_dpt['id_dep'].str.cat(df_dpt['libDep'], sep = ' - ')

dpt = df_dpt['dep_lib'].drop_duplicates().sort_values()

dpt_id_selected = '14'

file_path_dpt_resultats = '/kaggle/working/dpt/data/resultats_' + dpt_id_selected + '.csv'
dpt_resultats = pd.read_csv(file_path_dpt_resultats,low_memory=False)

# a. Résultats - Stats descriptives - DPT
dpt_resultats_overview = dpt_resultats[['libDepartement', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
dpt_resultats_overview = dpt_resultats_overview.drop_duplicates()

# b. Résultats (top10) - DPT
dpt_resultats_details = dpt_resultats[['indicateur','valeur']]
dpt_resultats_details['id_candidat'] = dpt_resultats_details['indicateur'].str[-2:]
dpt_resultats_details['id_candidat'] = dpt_resultats_details['id_candidat'].str.strip()

dpt_resultats_details['indicateur'] = dpt_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
dpt_resultats_details['indicateur']= dpt_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)

dpt_resultats_details = dpt_resultats_details.pivot(index ='id_candidat', columns='indicateur', values='valeur')
dpt_resultats_details = pd.DataFrame(dpt_resultats_details.to_records())
dpt_resultats_details["Voix"] = dpt_resultats_details["Voix"].fillna(0).astype(float).round().astype(int)
dpt_resultats_details = dpt_resultats_details.nlargest(10, 'Voix')
dpt_resultats_details = dpt_resultats_details[['Nuance candidat', 'Voix','% Voix/exprimés','% Voix/inscrits']]
dpt_resultats_details = dpt_resultats_details.dropna(axis=0, subset=['Nuance candidat'])
dpt_resultats_details.head()

In [ ]:

# 2.Circonscriptions du département sélectionné

# a.Carte
#Read the HTML content from the file
html_content = read_html_file('/kaggle/working/circo/map/map_' + dpt_id_selected + '.html')

# b.Stats des circonscriptions du département sélectionné
file_path_circo_stats = '/kaggle/working/circo/data/stats_' + dpt_id_selected + '.csv'
df_stats_circo_selected = pd.read_csv(file_path_circo_stats,low_memory=False)

df_stats_circo_demo = df_stats_circo_selected[['Nom de la circonscription','Inscrit_22','pop_légal_19','pop_légal_13','tvar_pop','pop_pole_aav','pop_cour_aav','pop_horsaav','pop_urb','pop_rur_periu','pop_rur_non_periu','age_moyen','dec90','dec75','dec50','dec25','dec10']]
df_stats_circo_acti = df_stats_circo_selected[['Nom de la circonscription','actemp','actcho','inactret','inactetu','inactm14','inactaut','actemp_hom','actcho_hom','inactret_hom','inactetu_hom','inactm14_hom','inactaut_hom','actemp_fem','actcho_fem','inactret_fem','inactetu_fem','inactm14_fem','inactaut_fem']]
df_stats_circo_qualif = df_stats_circo_selected[['Nom de la circonscription','actdip_PEU','actdip_CAP','actdip_BAC','actdip_BAC2','actdip_BAC3','actdip_BAC5','actdip_BAC3P']]
df_stats_circo_secteur = df_stats_circo_selected[['Nom de la circonscription','act_agr','act_art','act_cad','act_int','act_emp','act_ouv','act_cho']]
df_stats_circo_log = df_stats_circo_selected[['Nom de la circonscription','log_res','log_sec','log_vac','proprio','locatai','gratuit','maison','ach90','mfuel']]
df_stats_circo_fam = df_stats_circo_selected[['Nom de la circonscription','men_seul','men_coupae','men_coupse','men_monop','men_sfam','men_seul_com','men_coupse_com','men_coupae_com','men_monop_com','men_complexe_com']]
df_stats_circo_mob = df_stats_circo_selected[['Nom de la circonscription','iranr_com','iranr_dep','iranr_fra','iranr_etr','mobresid','ilt_com','ilt_dep','ilt_fra','ilt_etr','mobtrav','modtrans_aucun','modtrans_pied','modtrans_velo','modtrans_moto','modtrans_voit','modtrans_commun']]
df_stats_circo_niv_vie = df_stats_circo_selected[['Nom de la circonscription','tx_pauvrete60_diff','nivvie_median_diff','part_pauvres_diff','part_modestes_diff','part_medians_diff','part_plutot_aises_diff','part_aises_diff','D1_diff','D9_diff','rpt_D9_D1_diff','tx_pauvrete60_diff_trageRF1','tx_pauvrete60_diff_trageRF2','tx_pauvrete60_diff_trageRF3','tx_pauvrete60_diff_trageRF4','tx_pauvrete60_diff_trageRF5','tx_pauvrete60_diff_trageRF6']]
df_stats_circo_rev = df_stats_circo_selected[['Nom de la circonscription','PACT','PPEN','PPAT','PPSOC','PIMPOT']]
df_stats_circo_autres = df_stats_circo_selected[['Nom de la circonscription','acc_ecole','acc_college','acc_lycee','acc_medecin','acc_dentiste','acc_pharmacie','part_eloig']]


In [ ]:
# c.Résultats des circonscriptions du département sélectionné
file_path_circo_resultats = '/kaggle/working/circo/data/resultats_' + dpt_id_selected + '.csv'
df_resultats_circo_selected = pd.read_csv(file_path_circo_resultats,low_memory=False)

df_resultats_circo_selected.head()

In [ ]:

    # i. Stats descriptives
circo_resultats_overview = df_resultats_circo_selected[['libCirco', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
circo_resultats_overview = circo_resultats_overview.drop_duplicates()


    # ii. Résultats (top10)
circo_resultats_details = df_resultats_circo_selected[['id_circo','libCirco','indicateur','valeur']]
circo_resultats_details['id_candidat'] = circo_resultats_details['indicateur'].str[-2:]
circo_resultats_details['id_candidat'] = circo_resultats_details['id_candidat'].str.strip()

circo_resultats_details['indicateur'] = circo_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
circo_resultats_details['indicateur']= circo_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)

In [ ]:
groups = circo_resultats_details.groupby('id_circo')
for name,group in groups:
    tmp_details_circo = group.pivot(index = ['id_circo','id_candidat'], columns='indicateur', values='valeur')
    tmp_details_circo = pd.DataFrame(tmp_details_circo.to_records())
    tmp_details_circo["Voix"] = tmp_details_circo["Voix"].fillna(0).astype(float).round().astype(int)
    tmp_details_circo = tmp_details_circo.nlargest(10, 'Voix')
    tmp_details_circo = tmp_details_circo[['id_candidat','Nuance candidat', 'Voix','% Voix/exprimés','% Voix/inscrits']]
    tmp_details_circo = tmp_details_circo.dropna(axis=0, subset=['Nuance candidat'])
    print(tmp_details_circo)

In [ ]:
### d.Liste des circonscriptions du département sélectionné
df_circo = df[df['id_dep'] == dpt_id_selected].drop(columns=['codeCommune','nomCommune','numeroBureauVote','codeBureauVote','id_bv','libBv']).drop_duplicates()
df_circo['id_circo'] = df_circo['id_circo'].astype(str)
df_circo.sort_values(by='id_circo',inplace=True)
df_circo['circo_lib'] = df_circo['id_circo'].str.cat(df_circo['libCirco'], sep = ' - ')

circo =  df_circo['circo_lib'].drop_duplicates().sort_values()


circo_selected ='1400001 - 1ère circonscription'
#circo_selected = st.sidebar.selectbox('Sélection de la circonscription:', circo)
# ID circo
circo_id_selected = str(circo_selected).split(" - ")[0]


In [ ]:
# 3.Bureaux de votes de la circonscription sélectionnée

# a.Carte
# Read the HTML content from the file
html_content2 = read_html_file('/kaggle/working/bv/map/map_' + circo_id_selected + '.html')
# Display the HTML content in Streamlit

# b.Résultats des circonscriptions du département sélectionné
file_path_bv_resultats = '/kaggle/working/bv/data/resultats_' + circo_id_selected + '.csv'
df_resultats_bv_selected = pd.read_csv(file_path_bv_resultats,low_memory=False)
df_resultats_bv_selected.head()

In [ ]:
    # i. Stats descriptives
bv_resultats_overview = df_resultats_bv_selected[['id_bv','libBv', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
bv_resultats_overview = bv_resultats_overview.drop_duplicates()
bv_resultats_overview.head()

 


In [ ]:
   # ii. Résultats (top10)
bv_resultats_details = df_resultats_bv_selected[['id_bv','libBv','indicateur','valeur']]
bv_resultats_details['id_candidat'] = bv_resultats_details['indicateur'].str[-2:]
bv_resultats_details['id_candidat'] = bv_resultats_details['id_candidat'].str.strip()

bv_resultats_details['indicateur'] = bv_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
bv_resultats_details['indicateur']= bv_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)

#bv_resultats_details.head()

bv =  bv_resultats_details['id_bv'].drop_duplicates().sort_values()
bv

In [ ]:
#bv_selected = st.sidebar.selectbox('Sélection du bureau de vote:', bv)
# ID BV
bv_id_selected = '14026_0001'

df_bv = bv_resultats_details[bv_resultats_details['id_bv'] == bv_id_selected]

tmp_details_bv = df_bv.pivot(index = ['id_bv','id_candidat'], columns='indicateur', values='valeur')
tmp_details_bv = pd.DataFrame(tmp_details_bv.to_records())
tmp_details_bv["Voix"] = tmp_details_bv["Voix"].fillna(0).astype(float).round().astype(int)
tmp_details_bv = tmp_details_bv.nlargest(10, 'Voix')
tmp_details_bv = tmp_details_bv[['id_bv','id_candidat','Nuance candidat', 'Voix','% Voix/exprimés','% Voix/inscrits']]
tmp_details_bv.head()

In [ ]:
%%writefile app.py
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import streamlit as st
st.set_page_config(layout="wide")

# Load the HTML file
def read_html_file(filename):
    with open(filename, 'r') as f:
        return f.read()


# Stats definitions
df_stats_def = pd.read_csv('/kaggle/input/stats-def-csv/stats_def.csv',sep=';')
df_stats_def_demo = df_stats_def[(df_stats_def['Catégorie'] == 'Démographie') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_acti = df_stats_def[(df_stats_def['Catégorie'] == 'Activité') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_qualif = df_stats_def[(df_stats_def['Catégorie'] == 'Qualification') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_secteur = df_stats_def[(df_stats_def['Catégorie'] == 'Secteur') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_log = df_stats_def[(df_stats_def['Catégorie'] == 'Logements') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_fam = df_stats_def[(df_stats_def['Catégorie'] == 'Famille') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_mob = df_stats_def[(df_stats_def['Catégorie'] == 'Mobilité') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_rev = df_stats_def[(df_stats_def['Catégorie'] == 'Revenus') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_niv_vie = df_stats_def[(df_stats_def['Catégorie'] == 'Niveau de vie') | (df_stats_def['Variable'] == 'Nom de la circonscription')]
df_stats_def_autres = df_stats_def[(df_stats_def['Catégorie'] == 'Autres') | (df_stats_def['Variable'] == 'Nom de la circonscription')]

    
# 1.Liste des départements
file_path = '/kaggle/working/dataset_dpt_circo_bv_test.csv'
df = pd.read_csv(file_path,low_memory=False)
df.rename(columns={'codeDepartement': 'id_dep','nomDepartement': 'dep_name'}, inplace=True)
df['id_dep'] = df['id_dep'].astype(str)
df['id_circo'] = df['id_circo'].astype(str)
df['id_bv'] = df['id_bv'].astype(str)

df_dpt = df.drop(columns=['id_circo','libCirco','codeCommune','nomCommune','numeroBureauVote','codeBureauVote','id_bv','libBv']).drop_duplicates()
df_dpt['id_dep'] = df_dpt['id_dep'].astype(str)
df_dpt.sort_values(by='id_dep',inplace=True)
df_dpt['id_dep'] = df_dpt['id_dep'].astype(str)
df_dpt['dep_lib'] = df_dpt['id_dep'].str.cat(df_dpt['libDep'], sep = ' - ')

dpt = df_dpt['dep_lib'].drop_duplicates().sort_values()
dpt_selected = st.sidebar.selectbox('Sélection du département:', dpt)
# ID dpt
dpt_id_selected = dpt_selected.split(" - ")[0]

#dpt_id_selected = '14'

file_path_dpt_resultats = '/kaggle/working/dpt/data/resultats_' + dpt_id_selected + '.csv'
dpt_resultats = pd.read_csv(file_path_dpt_resultats,low_memory=False)

# a. Résultats - Stats descriptives - DPT
dpt_resultats_overview = dpt_resultats[['libDepartement', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
dpt_resultats_overview = dpt_resultats_overview.drop_duplicates()

data_container = st.container()
with data_container:
    st.write("Département - Elections législatives:")
    st.dataframe(dpt_resultats_overview,hide_index=True)

# b. Résultats (top10) - DPT
dpt_resultats_details = dpt_resultats[['indicateur','valeur']]
dpt_resultats_details['id_candidat'] = dpt_resultats_details['indicateur'].str[-2:]
dpt_resultats_details['id_candidat'] = dpt_resultats_details['id_candidat'].str.strip()

dpt_resultats_details['indicateur'] = dpt_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
dpt_resultats_details['indicateur']= dpt_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)

dpt_resultats_details = dpt_resultats_details.pivot(index ='id_candidat', columns='indicateur', values='valeur')
dpt_resultats_details = pd.DataFrame(dpt_resultats_details.to_records())
dpt_resultats_details["Voix"] = dpt_resultats_details["Voix"].fillna(0).astype(float).round().astype(int)
dpt_resultats_details = dpt_resultats_details.nlargest(10, 'Voix')
dpt_resultats_details = dpt_resultats_details[['Nuance candidat', 'Voix','% Voix/exprimés','% Voix/inscrits']]
dpt_resultats_details = dpt_resultats_details.dropna(axis=0, subset=['Nuance candidat'])

data_container2 = st.container()
with data_container2:
    st.write("Département - Résultats législatives (top10):")
    st.dataframe(dpt_resultats_details,hide_index=True)

    
# 2.Circonscriptions du département sélectionné

# a.Carte
#Read the HTML content from the file
html_content = read_html_file('/kaggle/working/circo/map/map_' + dpt_id_selected + '.html')
# Display the HTML content in Streamlit
map_container1 = st.container()

with map_container1:
    st.write("Circonscriptions:")
    st.components.v1.html(html_content,height=500)

# b.Stats des circonscriptions du département sélectionné
file_path_circo_stats = '/kaggle/working/circo/data/stats_' + dpt_id_selected + '.csv'
df_stats_circo_selected = pd.read_csv(file_path_circo_stats,low_memory=False)

df_stats_circo_demo = df_stats_circo_selected[['Nom de la circonscription','Inscrit_22','pop_légal_19','pop_légal_13','tvar_pop','pop_pole_aav','pop_cour_aav','pop_horsaav','pop_urb','pop_rur_periu','pop_rur_non_periu','age_moyen','dec90','dec75','dec50','dec25','dec10']]
df_stats_circo_acti = df_stats_circo_selected[['Nom de la circonscription','actemp','actcho','inactret','inactetu','inactm14','inactaut','actemp_hom','actcho_hom','inactret_hom','inactetu_hom','inactm14_hom','inactaut_hom','actemp_fem','actcho_fem','inactret_fem','inactetu_fem','inactm14_fem','inactaut_fem']]
df_stats_circo_qualif = df_stats_circo_selected[['Nom de la circonscription','actdip_PEU','actdip_CAP','actdip_BAC','actdip_BAC2','actdip_BAC3','actdip_BAC5','actdip_BAC3P']]
df_stats_circo_secteur = df_stats_circo_selected[['Nom de la circonscription','act_agr','act_art','act_cad','act_int','act_emp','act_ouv','act_cho']]
df_stats_circo_log = df_stats_circo_selected[['Nom de la circonscription','log_res','log_sec','log_vac','proprio','locatai','gratuit','maison','ach90','mfuel']]
df_stats_circo_fam = df_stats_circo_selected[['Nom de la circonscription','men_seul','men_coupae','men_coupse','men_monop','men_sfam','men_seul_com','men_coupse_com','men_coupae_com','men_monop_com','men_complexe_com']]
df_stats_circo_mob = df_stats_circo_selected[['Nom de la circonscription','iranr_com','iranr_dep','iranr_fra','iranr_etr','mobresid','ilt_com','ilt_dep','ilt_fra','ilt_etr','mobtrav','modtrans_aucun','modtrans_pied','modtrans_velo','modtrans_moto','modtrans_voit','modtrans_commun']]
df_stats_circo_niv_vie = df_stats_circo_selected[['Nom de la circonscription','tx_pauvrete60_diff','nivvie_median_diff','part_pauvres_diff','part_modestes_diff','part_medians_diff','part_plutot_aises_diff','part_aises_diff','D1_diff','D9_diff','rpt_D9_D1_diff','tx_pauvrete60_diff_trageRF1','tx_pauvrete60_diff_trageRF2','tx_pauvrete60_diff_trageRF3','tx_pauvrete60_diff_trageRF4','tx_pauvrete60_diff_trageRF5','tx_pauvrete60_diff_trageRF6']]
df_stats_circo_rev = df_stats_circo_selected[['Nom de la circonscription','PACT','PPEN','PPAT','PPSOC','PIMPOT']]
df_stats_circo_autres = df_stats_circo_selected[['Nom de la circonscription','acc_ecole','acc_college','acc_lycee','acc_medecin','acc_dentiste','acc_pharmacie','part_eloig']]

with st.expander("Statistiques descriptives:"):
    tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9, tab10 = st.tabs(["Démographie", "Activité/inactivité:", "Qualification","Secteurs","Logement","Famille","Mobilité","Niveau de vie","Revenus","Autres" ])
    
    with tab1:
        st.dataframe(df_stats_circo_demo,hide_index=True)
        st.dataframe(df_stats_def_demo,hide_index=True)
    with tab2:  
        st.dataframe(df_stats_circo_acti,hide_index=True)
        st.dataframe(df_stats_def_acti,hide_index=True)
    with tab3: 
        st.dataframe(df_stats_circo_qualif,hide_index=True)
        st.dataframe(df_stats_def_qualif,hide_index=True)
    with tab4:
        st.dataframe(df_stats_circo_secteur,hide_index=True)
        st.dataframe(df_stats_def_secteur,hide_index=True)
    with tab5:  
        st.dataframe(df_stats_circo_log,hide_index=True)
        st.dataframe(df_stats_def_log,hide_index=True)
    with tab6:
        st.dataframe(df_stats_circo_fam,hide_index=True)
        st.dataframe(df_stats_def_fam,hide_index=True)
    with tab7: 
        st.dataframe(df_stats_circo_mob,hide_index=True)
        st.dataframe(df_stats_def_mob,hide_index=True)
    with tab8: 
        st.dataframe(df_stats_circo_niv_vie,hide_index=True)
        st.dataframe(df_stats_def_niv_vie,hide_index=True)
    with tab9:  
        st.dataframe(df_stats_circo_rev,hide_index=True)
        st.dataframe(df_stats_def_rev,hide_index=True)
    with tab10:
        st.dataframe(df_stats_circo_autres,hide_index=True)
        st.dataframe(df_stats_def_autres,hide_index=True)

# c.Résultats des circonscriptions du département sélectionné
file_path_circo_resultats = '/kaggle/working/circo/data/resultats_' + dpt_id_selected + '.csv'
df_resultats_circo_selected = pd.read_csv(file_path_circo_resultats,low_memory=False)

    # i. Stats descriptives
circo_resultats_overview = df_resultats_circo_selected[['libCirco', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
circo_resultats_overview = circo_resultats_overview.drop_duplicates()
data_container3 = st.container()
with data_container3:
    st.write("Circonscriptions - Elections législatives:")
    st.dataframe(circo_resultats_overview,hide_index=True)

    # ii. Résultats (top10)
circo_resultats_details = df_resultats_circo_selected[['id_circo','libCirco','indicateur','valeur']]
circo_resultats_details['id_candidat'] = circo_resultats_details['indicateur'].str[-2:]
circo_resultats_details['id_candidat'] = circo_resultats_details['id_candidat'].str.strip()

circo_resultats_details['indicateur'] = circo_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
circo_resultats_details['indicateur']= circo_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)

#circo_resultats_details

with st.expander("Circonscriptions - Résultats législatives (top10):"):
    groups = circo_resultats_details.groupby('id_circo')
    for name,group in groups:
        tmp_details_circo = group.pivot(index = ['id_circo','id_candidat'], columns='indicateur', values='valeur')
        tmp_details_circo = pd.DataFrame(tmp_details_circo.to_records())
        tmp_details_circo["Voix"] = tmp_details_circo["Voix"].fillna(0).astype(float).round().astype(int)
        tmp_details_circo = tmp_details_circo.nlargest(10, 'Voix')
        tmp_details_circo = tmp_details_circo[['id_candidat','Nuance candidat', 'Voix','% Voix/exprimés','% Voix/inscrits']]
        tmp_details_circo = tmp_details_circo.dropna(axis=0, subset=['Nuance candidat'])
        st.write(name)
        st.dataframe(tmp_details_circo,hide_index=True)   

# d.Liste des circonscriptions du département sélectionné
df_circo = df[df['id_dep'] == dpt_id_selected].drop(columns=['codeCommune','nomCommune','numeroBureauVote','codeBureauVote','id_bv','libBv']).drop_duplicates()
df_circo['id_circo'] = df_circo['id_circo'].astype(str)
df_circo.sort_values(by='id_circo',inplace=True)
df_circo['circo_lib'] = df_circo['id_circo'].str.cat(df_circo['libCirco'], sep = ' - ')

circo =  df_circo['circo_lib'].drop_duplicates().sort_values()
circo_selected = st.sidebar.selectbox('Sélection de la circonscription:', circo)
# ID circo
circo_id_selected = str(circo_selected).split(" - ")[0]

# 3.Bureaux de votes de la circonscription sélectionnée

# a.Carte
# Read the HTML content from the file
html_content2 = read_html_file('/kaggle/working/bv/map/map_' + circo_id_selected + '.html')
# Display the HTML content in Streamlit
map_container2 = st.container()
with map_container2:
    st.write("Bureaux de vote:")
    st.components.v1.html(html_content2,height=500)

# b.Résultats des circonscriptions du département sélectionné
file_path_bv_resultats = '/kaggle/working/bv/data/resultats_' + circo_id_selected + '.csv'
df_resultats_bv_selected = pd.read_csv(file_path_bv_resultats,low_memory=False)

    # i. Stats descriptives
bv_resultats_overview = df_resultats_bv_selected[['id_bv','libBv', 'Inscrits', 'Votants', '% Votants', 'Abstentions', '% Abstentions', 'Exprimés', '% Exprimés/inscrits', '% Exprimés/votants', 'Blancs', '% Blancs/inscrits', '% Blancs/votants', 'Nuls', '% Nuls/inscrits', '% Nuls/votants']]
bv_resultats_overview = bv_resultats_overview.drop_duplicates()
data_container3 = st.container()
with data_container3:
    st.write("Bureaux de votes - Elections législatives:")
    st.dataframe(bv_resultats_overview,hide_index=True)

    # ii. Résultats (top10)
bv_resultats_details = df_resultats_bv_selected[['id_bv','libBv','indicateur','valeur']]
bv_resultats_details['id_candidat'] = bv_resultats_details['indicateur'].str[-2:]
bv_resultats_details['id_candidat'] = bv_resultats_details['id_candidat'].str.strip()

bv_resultats_details['indicateur'] = bv_resultats_details['indicateur'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
bv_resultats_details['indicateur']= bv_resultats_details['indicateur'].apply(lambda x: x[:-1] if isinstance(x, str) else x)



bv =  bv_resultats_details['id_bv'].drop_duplicates().sort_values()
bv_selected = st.sidebar.selectbox('Sélection du bureau de vote:', bv)
# ID BV
bv_id_selected = bv_selected

df_bv = bv_resultats_details[bv_resultats_details['id_bv'] == bv_id_selected]

tmp_details_bv = df_bv.pivot(index = ['id_bv','id_candidat'], columns='indicateur', values='valeur')
tmp_details_bv = pd.DataFrame(tmp_details_bv.to_records())
tmp_details_bv["Voix"] = tmp_details_bv["Voix"].fillna(0).astype(float).round().astype(int)
tmp_details_bv = tmp_details_bv.nlargest(10, 'Voix')
tmp_details_bv = tmp_details_bv[['id_bv','id_candidat','Nuance candidat', 'Voix','% Voix/exprimés','% Voix/inscrits']]
tmp_details_bv = tmp_details_bv.dropna(axis=0, subset=['Nuance candidat'])

data_container4 = st.container()
with data_container4:
    st.write("Bureau de vote sélectionné - Résultats législatives (top10):")
    st.dataframe(tmp_details_bv,hide_index=True)

In [ ]:
!npm install localtunnel

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501